In [11]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [13]:
qdrant_client = QdrantClient(
    url=os.environ.get("QDRANT_CLUSTER_URL"),
    api_key=os.environ.get("QDRANT_API_KEY"),
)

In [14]:
def create_collection(name: str, vector_size: int = 1536) -> bool:
    return qdrant_client.recreate_collection(
        collection_name=name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )

In [15]:
status = create_collection("test_collection")

In [16]:
status

True